In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
##Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(columns=['CustomerId','RowNumber','Surname'],axis=1)

In [4]:
data.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [5]:
##Encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


[['Geography']] → keeps it as 2D (DataFrame).


['Geography'] → just a 1D Series.

In [7]:
#One hot encode 'Geography'

from sklearn.preprocessing import OneHotEncoder

onehot_geography = OneHotEncoder()

geo_encoder = onehot_geography.fit_transform(data[['Geography']])

geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
onehot_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_geography.get_feature_names_out(['Geography']))

In [11]:
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [12]:
##Combine the One hot encoder columns with the original data

df = pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
##Save the encoder and scaler 

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

In [14]:
##Divide the Dataset into X,y

X = df.drop('Exited',axis=1)
y = df['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

##Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [16]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [17]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard  #TensorBoard is a visualization tool for TensorFlow (and Keras).
import datetime

In [18]:
##Build Our ANN Model

#Dense layer = fully connected layer.
#64 = number of neurons (units) in this layer.
#X_train.shape[1] = number of features in your training data.

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  ##HL1 connected to input layer

    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') #output layer
]
)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [21]:
## Compile the model

model.compile(optimizer=opt,loss= loss,metrics=['accuracy'])

In [22]:
##Set up the TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
##Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [24]:
##Train the model

history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 2ms/step - loss: 0.3964 - accuracy: 0.8324 - val_loss: 0.3633 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3537 - accuracy: 0.8537 - val_loss: 0.3535 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3485 - accuracy: 0.8565 - val_loss: 0.3537 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3420 - accuracy: 0.8600 - val_loss: 0.3366 - val_accuracy: 0.8570
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3400 - accuracy: 0.8609 - val_loss: 0.3445 - val_accuracy: 0.8605
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3402 - accuracy: 0.8597 - val_loss: 0.3522 - val_accuracy: 0.8550
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3329 - accuracy: 0.8631 - val_loss: 0.3362 - val_accuracy: 0.86

In [26]:
model.save('model.h5')

In [27]:
##Load the Tensorboard Extention
%load_ext tensorboard

In [28]:
%tensorboard --logdir logs/fit20250905-111102

Reusing TensorBoard on port 6007 (pid 23344), started 3 days, 3:57:19 ago. (Use '!kill 23344' to kill it.)